In [1]:
import networkx as nx
from pickle import *
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
import yaml
from datascience import *

In [2]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

### Congress on Twitter data, version 2

The original Congress on Twitter data was difficult to link to biographical information about each legislator.

In order to address this issue, I am releasing this second version of the Congress on Twitter dataset. I recommend that you use this dataset instead of the previous one, but you are free to choose either one.

The main differences between this dataset and the previous one are:

1. This dataset only consists of 'official' Twitter accounts for each legislator (many legislators also have distinct campaign-related accounts)
2. This dataset has been linked to the biographical information about each legislator, so it has information like party, state, et cetera

In order to narrow the data down to official Twitter accounts, and to merge it with the biographical information, I had to omit

* 16 legislators for whom I did not have official Twitter account info
* 3 legislators whose official Twitter accounts didn't follow any other Members of Congress (so they were isolates in the network)

#### Reading the data in

In [3]:
official_congress_twitter, official_congress_friends, legislator_data = load(open('congress_twitter_v2.pickle', 'rb'))
legislator_data = Table().from_df(legislator_data)

#### Network of official legislator Twitter accounts

In [4]:
official_congress_twitter.number_of_nodes()

519

In [5]:
nx.average_clustering(official_congress_twitter)

0.6698933484362136

In [6]:
nx.attribute_assortativity_coefficient(official_congress_twitter, 'party')

0.5984762733850006

In [7]:
official_congress_twitter.nodes(data=True)

[(2482022082,
  {'address': '1728 Longworth HOB; Washington DC 20515-0913',
   'bioguide': 'J000296',
   'birthday': '1972-10-31',
   'caucus': nan,
   'class': nan,
   'contact_form': 'https://jolly.house.gov/contact/email-me',
   'district': 13.0,
   'end': '2017-01-03',
   'fax': '202-225-9764',
   'first': 'David',
   'gender': 'M',
   'last': 'Jolly',
   'middle': 'W.',
   'nickname': nan,
   'office': '1728 Longworth House Office Building',
   'official_full': 'David W. Jolly',
   'party': 'Republican',
   'phone': '202-225-5961',
   'religion': nan,
   'rss_url': nan,
   'start': '2015-01-06',
   'state': 'FL',
   'state_rank': nan,
   'suffix': nan,
   'twitter': 2482022082.0,
   'type': 'rep',
   'url': 'http://jolly.house.gov'}),
 (2974648323,
  {'address': '1023 Longworth HOB; Washington DC 20515-0525',
   'bioguide': 'K000387',
   'birthday': '1966-12-17',
   'caucus': nan,
   'class': nan,
   'contact_form': nan,
   'district': 25.0,
   'end': '2017-01-03',
   'fax': '202-

#### Legislator biographical Info

In [8]:
legislator_data

twitter,first,last,middle,nickname,official_full,suffix,address,caucus,class,contact_form,district,end,fax,office,party,phone,rss_url,start,state,state_rank,type,url,birthday,gender,religion,twitter_id,bioguide
sensherrodbrown,Sherrod,Brown,nan,nan,Sherrod Brown,nan,713 Hart Senate Office Building Washington DC 20510,nan,1,http://www.brown.senate.gov/contact,nan,2019-01-03,202-228-6321,713 Hart Senate Office Building,Democrat,202-224-2315,http://www.brown.senate.gov/rss/feeds/?type=all&,2013-01-03,OH,senior,sen,http://www.brown.senate.gov,1952-11-09,M,Lutheran,4.39108e+07,B000944
senatorcantwell,Maria,Cantwell,nan,nan,Maria Cantwell,nan,511 Hart Senate Office Building Washington DC 20510,nan,1,http://www.cantwell.senate.gov/public/index.cfm/email-maria,nan,2019-01-03,202-228-0514,511 Hart Senate Office Building,Democrat,202-224-3441,http://www.cantwell.senate.gov/public/index.cfm/rss/feed,2013-01-03,WA,junior,sen,https://www.cantwell.senate.gov,1958-10-13,F,Roman Catholic,1.17502e+08,C000127
senatorcardin,Benjamin,Cardin,L.,nan,Benjamin L. Cardin,nan,509 Hart Senate Office Building Washington DC 20510,nan,1,http://www.cardin.senate.gov/contact/,nan,2019-01-03,202-224-1651,509 Hart Senate Office Building,Democrat,202-224-4524,http://www.cardin.senate.gov/rss/feeds/?type=all,2013-01-03,MD,junior,sen,https://www.cardin.senate.gov,1943-10-05,M,Jewish,1.09071e+08,C000141
senatorcarper,Thomas,Carper,Richard,nan,Thomas R. Carper,nan,513 Hart Senate Office Building Washington DC 20510,nan,1,http://www.carper.senate.gov/public/index.cfm/email-sena ...,nan,2019-01-03,202-228-2190,513 Hart Senate Office Building,Democrat,202-224-2441,http://www.carper.senate.gov/public/index.cfm/rss/feed,2013-01-03,DE,senior,sen,http://www.carper.senate.gov/public,1947-01-23,M,Presbyterian,2.49788e+08,C000174
senbobcasey,Robert,Casey,P.,Bob,"Robert P. Casey, Jr.",Jr.,393 Russell Senate Office Building Washington DC 20510,nan,1,http://www.casey.senate.gov/contact/,nan,2019-01-03,202-228-0604,393 Russell Senate Office Building,Democrat,202-224-6324,http://www.casey.senate.gov/rss/feeds/?all,2013-01-03,PA,senior,sen,https://www.casey.senate.gov,1960-04-13,M,nan,1.71599e+08,C001070
senbobcorker,Bob,Corker,nan,nan,Bob Corker,nan,425 Dirksen Senate Office Building Washington DC 20510,nan,1,http://www.corker.senate.gov/public/index.cfm/emailme,nan,2019-01-03,202-228-0566,425 Dirksen Senate Office Building,Republican,202-224-3344,http://www.corker.senate.gov/public/index.cfm/rss/feed,2013-01-03,TN,junior,sen,http://www.corker.senate.gov,1952-08-24,M,nan,7.33038e+07,C001071
senfeinstein,Dianne,Feinstein,nan,nan,Dianne Feinstein,nan,331 Hart Senate Office Building Washington DC 20510,nan,1,https://www.feinstein.senate.gov/public/index.cfm/e-mail-me,nan,2019-01-03,202-228-3954,331 Hart Senate Office Building,Democrat,202-224-3841,http://www.feinstein.senate.gov/public/?a=rss.feed,2013-01-03,CA,senior,sen,http://www.feinstein.senate.gov,1933-06-22,F,Jewish,4.76257e+08,F000062
senorrinhatch,Orrin,Hatch,G.,nan,Orrin G. Hatch,nan,104 Hart Senate Office Building Washington DC 20510,nan,1,http://www.hatch.senate.gov/public/index.cfm/contact?p=E ...,nan,2019-01-03,202-224-6331,104 Hart Senate Office Building,Republican,202-224-5251,http://www.hatch.senate.gov/public/index.cfm/rss/feed,2013-01-03,UT,senior,sen,http://www.hatch.senate.gov,1934-03-22,M,Latter Day Saints,2.62757e+08,H000338
nan,Amy,Klobuchar,Jean,nan,Amy Klobuchar,nan,302 Hart Senate Office Building Washington DC 20510,nan,1,http://www.klobuchar.senate.gov/public/email-amy,nan,2019-01-03,202-228-2186,302 Hart Senate Office Building,Democrat,202-224-3244,nan,2013-01-03,MN,senior,sen,http://www.klobuchar.senate.gov,1960-05-25,F,nan,nan,K000367
mccaskilloffice,Claire,McCaskill,nan,nan,Claire McCaskill,nan,730 Hart Senate Office Building Washington DC 20510,nan,1,http://www.mccaskill.senate.gov/?p=contact,nan,2019-01-03,202-228-6326,730 Hart Senate Office Building,Democrat,202-224-6154,http://mccaskill.senate.gov

**Please note** that more information about the legislators can be joined into this info; see the file `prep_congress_moc` for more info on how to open up those data.